In [1]:
import pandas as pd

# 原始数据
df = pd.read_csv('y.csv')

# 编码公式：将实际值映射到 [-1, 0, +1]
df_encoded = df.copy()
df_encoded['x1c'] = (df['x1'] - 1.0) / 0.5  # 中心点1.0，步长0.5
df_encoded['x2c'] = (df['x2'] - 60) / 20    # 中心点60，步长20
df_encoded['x3c'] = (df['x3'] - 50) / 10    # 中心点50，步长10

print("编码后的设计矩阵：\n", df_encoded[['x1','x2','x3','x1c', 'x2c', 'x3c','y']].round(2))

编码后的设计矩阵：
      x1  x2  x3  x1c  x2c  x3c     y
0   1.0  80  40  0.0  1.0 -1.0  1476
1   1.0  60  50  0.0  0.0  0.0  1854
2   1.5  80  50  1.0  1.0  0.0  1265
3   1.0  60  50  0.0  0.0  0.0  1865
4   1.5  60  40  1.0  0.0 -1.0  1642
5   1.5  40  50  1.0 -1.0  0.0  1500
6   1.0  60  50  0.0  0.0  0.0  1887
7   1.0  60  50  0.0  0.0  0.0  1892
8   1.0  60  50  0.0  0.0  0.0  1839
9   1.0  80  60  0.0  1.0  1.0  1364
10  0.5  80  50 -1.0  1.0  0.0  1333
11  0.5  60  60 -1.0  0.0  1.0  1543
12  0.5  40  50 -1.0 -1.0  0.0  1353
13  0.5  60  40 -1.0  0.0 -1.0  1619
14  1.0  40  40  0.0 -1.0 -1.0  1443
15  1.5  60  60  1.0  0.0  1.0  1665
16  1.0  40  60  0.0 -1.0  1.0  1483


In [2]:
from sklearn.preprocessing import PolynomialFeatures

# 生成二次多项式特征（包含交互项和平方项）
poly = PolynomialFeatures(degree=2, include_bias=False)
# 编码值
X_poly1 = poly.fit_transform(df_encoded[['x1c', 'x2c', 'x3c']])
feature_names1 = poly.get_feature_names_out(['x1c', 'x2c', 'x3c'])
df_poly1 = pd.DataFrame(X_poly1, columns=feature_names1)
print("\n多项式特征矩阵：\n", df_poly1.head().round(2))
# 未编码值
X_poly2 = poly.fit_transform(df_encoded[['x1', 'x2', 'x3']])
feature_names2 = poly.get_feature_names_out(['x1', 'x2', 'x3'])
df_poly2 = pd.DataFrame(X_poly2, columns=feature_names2)
print("\n多项式特征矩阵：\n", df_poly2.head().round(2))


多项式特征矩阵：
    x1c  x2c  x3c  x1c^2  x1c x2c  x1c x3c  x2c^2  x2c x3c  x3c^2
0  0.0  1.0 -1.0    0.0      0.0     -0.0    1.0     -1.0    1.0
1  0.0  0.0  0.0    0.0      0.0      0.0    0.0      0.0    0.0
2  1.0  1.0  0.0    1.0      1.0      0.0    1.0      0.0    0.0
3  0.0  0.0  0.0    0.0      0.0      0.0    0.0      0.0    0.0
4  1.0  0.0 -1.0    1.0      0.0     -1.0    0.0     -0.0    1.0

多项式特征矩阵：
     x1    x2    x3  x1^2  x1 x2  x1 x3    x2^2   x2 x3    x3^2
0  1.0  80.0  40.0  1.00   80.0   40.0  6400.0  3200.0  1600.0
1  1.0  60.0  50.0  1.00   60.0   50.0  3600.0  3000.0  2500.0
2  1.5  80.0  50.0  2.25  120.0   75.0  6400.0  4000.0  2500.0
3  1.0  60.0  50.0  1.00   60.0   50.0  3600.0  3000.0  2500.0
4  1.5  60.0  40.0  2.25   90.0   60.0  3600.0  2400.0  1600.0


In [3]:
import statsmodels.api as sm

# 添加常数项并拟合模型
X_sm1 = sm.add_constant(df_poly1)
y = df['y']
model1 = sm.OLS(y, X_sm1)
results1 = model1.fit()
# 输出回归结果摘要
print("\n编码回归分析摘要：")
print(results1.summary())
# 添加常数项并拟合模型
X_sm2 = sm.add_constant(df_poly2)
model2 = sm.OLS(y, X_sm2)
results2 = model2.fit()
# 输出回归结果摘要
print("\n未编码回归分析摘要：")
print(results2.summary())


编码回归分析摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     91.83
Date:                Mon, 07 Apr 2025   Prob (F-statistic):           1.98e-06
Time:                        22:01:05   Log-Likelihood:                -74.183
No. Observations:                  17   AIC:                             168.4
Df Residuals:                       7   BIC:                             176.7
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1867.4000     13.246    140.9

e:\Python\lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=17 observations were given.
  return hypotest_fun_in(*args, **kwds)
e:\Python\lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=17 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [4]:
# 实际值模型评估与诊断
print(f"\nR²: {results2.rsquared:.4f}")
print(f"调整R²: {results2.rsquared_adj:.4f}")
print("\n系数估计与显著性：")
coeff_table = pd.DataFrame({
    '系数': results2.params.round(2),
    '标准误': results2.bse.round(2),
    't值': results2.tvalues.round(2),
    'p值': results2.pvalues.round(6)
})
print(coeff_table)


R²: 0.9916
调整R²: 0.9808

系数估计与显著性：
            系数     标准误     t值        p值
const -4489.70  468.36  -9.59  0.000028
x1     1446.60  208.81   6.93  0.000226
x2      114.80    5.91  19.43  0.000000
x3       90.59   15.43   5.87  0.000616
x1^2   -657.80   57.74 -11.39  0.000009
x1 x2    -5.37    1.48  -3.63  0.008404
x1 x3     4.95    2.96   1.67  0.138599
x2^2     -0.85    0.04 -23.57  0.000000
x2 x3    -0.19    0.07  -2.57  0.037228
x3^2     -0.86    0.14  -5.94  0.000577


In [5]:
# 多重共线性检查（VIF）
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 计算方差膨胀因子 (VIF)
vif_data = pd.DataFrame()
vif_data["特征"] = X_sm2.columns
vif_data["VIF"] = [variance_inflation_factor(X_sm2.values, i) for i in range(X_sm2.shape[1])]
print("\n方差膨胀因子 (VIF)：\n", vif_data)


方差膨胀因子 (VIF)：
       特征          VIF
0  const  4250.850000
1     x1    99.400000
2     x2   127.400000
3     x3   217.000000
4   x1^2    31.405882
5  x1 x2    27.000000
6  x1 x3    59.000000
7   x2^2    69.405882
8  x2 x3    69.000000
9   x3^2   191.005882


In [14]:
coefficients = results2.params  # 从回归结果中提取系数

# # 分离线性项和二次项系数
b0 = coefficients['const']
b_linear = coefficients[['x1', 'x2', 'x3']].values  # 线性项系数
b_quad = coefficients[['x1^2', 'x2^2', 'x3^2']].values  # 二次项系数
b_interaction = coefficients[['x1 x2', 'x1 x3', 'x2 x3']].values  # 交互项系数

In [15]:
# 构建矩阵b和B
import numpy as np

k = 3  # 因子数量
b = b_linear.reshape(-1, 1)  # 转为列向量

# 构建对称矩阵B
B = np.zeros((k, k))
np.fill_diagonal(B, b_quad)  # 对角线为二次项系数
B[0, 1] = B[1, 0] = b_interaction[0] / 2  # x1x2
B[0, 2] = B[2, 0] = b_interaction[1] / 2  # x1x3
B[1, 2] = B[2, 1] = b_interaction[2] / 2  # x2x3

In [17]:
# 求解驻点：xs = -0.5 * B^{-1} * b
B_inv = np.linalg.inv(B)
xs = -0.5 * np.dot(B_inv, b)

print("驻点坐标:\n", pd.DataFrame(xs, index=['x1', 'x2', 'x3'], columns=['Stationary Point']))

驻点坐标:
     Stationary Point
x1          1.045604
x2         58.673649
x3         49.367121


In [9]:
eigenvalues = np.linalg.eigvals(B)

if all(eig > 0 for eig in eigenvalues):
    print("驻点为最小值点")
elif all(eig < 0 for eig in eigenvalues):
    print("驻点为最大值点")
else:
    print("驻点为鞍点")

驻点为最大值点
